In [45]:
# use python2
'''
0429
Error analysis:
dictionary too few words
use word2vec to enrich our word list

using all round7 reviews as corpus
'''
import pandas as pd

# read
train = pd.read_table("/Users/luxiaopeng/notebook/inls613/closure_prediction/review_by_business.txt")
# test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
# unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
#  delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews" % (train["text"].size))



Read 2087 labeled train reviews


In [46]:
train.rename(columns={"text":"review"}, inplace=True)

In [48]:
train = train[["business_id","review"]]

In [52]:
# drop na
train = train.dropna()

In [54]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords = False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [55]:
'''
Next, we want a specific input format. Word2Vec expects single sentences, 
each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. 
There are all kinds of gotchas in natural language. English sentences can 
end with "?", "!", """, or ".", among other things, and spacing and 
capitalization are not reliable guides either. For this reason, 
we'll use NLTK's punkt tokenizer for sentence splitting. In order to 
use this, you will need to install NLTK and use nltk.download() to 
download the relevant training file for punkt.
'''
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download('popular')
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords = False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []
print('parsing sentences from training set')
for review in train['review']:
    sentences += review_to_sentences(review, tokenizer)

In [1]:
# check
print len(sentences)


NameError: name 'sentences' is not defined

In [14]:
import pickle


with open('sentence_5m.pkl', 'rb') as f:
    sentences = pickle.load(f)

In [15]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print('training model...')
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)
model_name = "yelp_300features_40minwords_10context"
model.save(model_name)

2018-04-30 19:31:26,407 : INFO : 'pattern' package not found; tag filters are not available for English
2018-04-30 19:31:26,429 : INFO : collecting all words and their counts
2018-04-30 19:31:26,432 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


training model...


2018-04-30 19:31:27,742 : INFO : PROGRESS: at sentence #10000, processed 138382 words, keeping 8055 word types
2018-04-30 19:31:29,349 : INFO : PROGRESS: at sentence #20000, processed 285664 words, keeping 11844 word types
2018-04-30 19:31:30,389 : INFO : PROGRESS: at sentence #30000, processed 437188 words, keeping 14869 word types
2018-04-30 19:31:31,011 : INFO : PROGRESS: at sentence #40000, processed 578498 words, keeping 17240 word types
2018-04-30 19:31:31,634 : INFO : PROGRESS: at sentence #50000, processed 718641 words, keeping 19088 word types
2018-04-30 19:31:32,622 : INFO : PROGRESS: at sentence #60000, processed 861642 words, keeping 20598 word types
2018-04-30 19:31:34,538 : INFO : PROGRESS: at sentence #70000, processed 1001683 words, keeping 22011 word types
2018-04-30 19:31:35,680 : INFO : PROGRESS: at sentence #80000, processed 1147426 words, keeping 23480 word types
2018-04-30 19:31:36,492 : INFO : PROGRESS: at sentence #90000, processed 1292174 words, keeping 24971 w

2018-04-30 19:33:06,575 : INFO : PROGRESS: at sentence #730000, processed 10371563 words, keeping 67003 word types
2018-04-30 19:33:08,743 : INFO : PROGRESS: at sentence #740000, processed 10513433 words, keeping 67406 word types
2018-04-30 19:33:09,601 : INFO : PROGRESS: at sentence #750000, processed 10656739 words, keeping 67781 word types
2018-04-30 19:33:11,266 : INFO : PROGRESS: at sentence #760000, processed 10794821 words, keeping 68146 word types
2018-04-30 19:33:13,376 : INFO : PROGRESS: at sentence #770000, processed 10937652 words, keeping 68538 word types
2018-04-30 19:33:14,815 : INFO : PROGRESS: at sentence #780000, processed 11085211 words, keeping 69033 word types
2018-04-30 19:33:16,977 : INFO : PROGRESS: at sentence #790000, processed 11229342 words, keeping 69406 word types
2018-04-30 19:33:18,284 : INFO : PROGRESS: at sentence #800000, processed 11368953 words, keeping 69743 word types
2018-04-30 19:33:19,095 : INFO : PROGRESS: at sentence #810000, processed 115092

2018-04-30 19:35:05,940 : INFO : PROGRESS: at sentence #1440000, processed 20407767 words, keeping 90909 word types
2018-04-30 19:35:08,743 : INFO : PROGRESS: at sentence #1450000, processed 20550295 words, keeping 91151 word types
2018-04-30 19:35:10,407 : INFO : PROGRESS: at sentence #1460000, processed 20702078 words, keeping 91519 word types
2018-04-30 19:35:13,519 : INFO : PROGRESS: at sentence #1470000, processed 20844886 words, keeping 91793 word types
2018-04-30 19:35:15,677 : INFO : PROGRESS: at sentence #1480000, processed 20990907 words, keeping 92023 word types
2018-04-30 19:35:18,361 : INFO : PROGRESS: at sentence #1490000, processed 21123968 words, keeping 92327 word types
2018-04-30 19:35:19,706 : INFO : PROGRESS: at sentence #1500000, processed 21266553 words, keeping 92635 word types
2018-04-30 19:35:23,182 : INFO : PROGRESS: at sentence #1510000, processed 21409681 words, keeping 92913 word types
2018-04-30 19:35:25,456 : INFO : PROGRESS: at sentence #1520000, process

...

2018-04-30 20:08:49,923 : INFO : EPOCH 5 - PROGRESS: at 24.50% examples, 106433 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:08:50,951 : INFO : EPOCH 5 - PROGRESS: at 24.58% examples, 105895 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:08:51,987 : INFO : EPOCH 5 - PROGRESS: at 24.78% examples, 105840 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:08:53,056 : INFO : EPOCH 5 - PROGRESS: at 25.07% examples, 106183 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:08:54,059 : INFO : EPOCH 5 - PROGRESS: at 25.31% examples, 106333 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:08:55,070 : INFO : EPOCH 5 - PROGRESS: at 25.65% examples, 106888 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:08:56,072 : INFO : EPOCH 5 - PROGRESS: at 25.97% examples, 107328 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:08:57,087 : INFO : EPOCH 5 - PROGRESS: at 26.21% examples, 107457 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:08:58,165 : INFO : EPOCH 5 - PROGRESS: at 26.55% examples, 107930 words/s, in_qsiz

2018-04-30 20:10:07,596 : INFO : EPOCH 5 - PROGRESS: at 40.30% examples, 106305 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:10:08,794 : INFO : EPOCH 5 - PROGRESS: at 40.43% examples, 106001 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:10:09,971 : INFO : EPOCH 5 - PROGRESS: at 40.55% examples, 105711 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:11,009 : INFO : EPOCH 5 - PROGRESS: at 40.75% examples, 105680 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:12,033 : INFO : EPOCH 5 - PROGRESS: at 41.06% examples, 105945 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:13,038 : INFO : EPOCH 5 - PROGRESS: at 41.37% examples, 106221 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:14,159 : INFO : EPOCH 5 - PROGRESS: at 41.65% examples, 106363 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:15,194 : INFO : EPOCH 5 - PROGRESS: at 41.96% examples, 106616 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:10:16,400 : INFO : EPOCH 5 - PROGRESS: at 42.27% examples, 106778 words/s, in_qsiz

2018-04-30 20:11:26,566 : INFO : EPOCH 5 - PROGRESS: at 55.66% examples, 105081 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:11:27,601 : INFO : EPOCH 5 - PROGRESS: at 55.98% examples, 105300 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:11:28,664 : INFO : EPOCH 5 - PROGRESS: at 56.21% examples, 105322 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:11:29,672 : INFO : EPOCH 5 - PROGRESS: at 56.49% examples, 105442 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:11:30,680 : INFO : EPOCH 5 - PROGRESS: at 56.71% examples, 105484 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:11:31,770 : INFO : EPOCH 5 - PROGRESS: at 56.96% examples, 105520 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:11:32,777 : INFO : EPOCH 5 - PROGRESS: at 57.27% examples, 105717 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:11:33,857 : INFO : EPOCH 5 - PROGRESS: at 57.61% examples, 105940 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:11:35,116 : INFO : EPOCH 5 - PROGRESS: at 57.89% examples, 105963 words/s, in_qsiz

2018-04-30 20:12:43,743 : INFO : EPOCH 5 - PROGRESS: at 75.23% examples, 111063 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:12:44,758 : INFO : EPOCH 5 - PROGRESS: at 75.57% examples, 111241 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:12:45,899 : INFO : EPOCH 5 - PROGRESS: at 75.88% examples, 111359 words/s, in_qsize 7, out_qsize 1
2018-04-30 20:12:46,951 : INFO : EPOCH 5 - PROGRESS: at 76.07% examples, 111319 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:12:47,967 : INFO : EPOCH 5 - PROGRESS: at 76.40% examples, 111495 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:12:48,998 : INFO : EPOCH 5 - PROGRESS: at 76.71% examples, 111624 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:12:50,003 : INFO : EPOCH 5 - PROGRESS: at 77.02% examples, 111781 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:12:51,034 : INFO : EPOCH 5 - PROGRESS: at 77.30% examples, 111868 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:12:52,061 : INFO : EPOCH 5 - PROGRESS: at 77.52% examples, 111875 words/s, in_qsiz

2018-04-30 20:14:01,858 : INFO : EPOCH 5 - PROGRESS: at 94.42% examples, 114096 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:14:02,882 : INFO : EPOCH 5 - PROGRESS: at 94.68% examples, 114146 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:14:03,904 : INFO : EPOCH 5 - PROGRESS: at 94.99% examples, 114265 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:14:04,968 : INFO : EPOCH 5 - PROGRESS: at 95.32% examples, 114390 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:14:06,015 : INFO : EPOCH 5 - PROGRESS: at 95.58% examples, 114435 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:14:07,080 : INFO : EPOCH 5 - PROGRESS: at 95.82% examples, 114468 words/s, in_qsize 7, out_qsize 0
2018-04-30 20:14:08,085 : INFO : EPOCH 5 - PROGRESS: at 96.14% examples, 114579 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:14:09,108 : INFO : EPOCH 5 - PROGRESS: at 96.43% examples, 114678 words/s, in_qsize 8, out_qsize 0
2018-04-30 20:14:10,111 : INFO : EPOCH 5 - PROGRESS: at 96.74% examples, 114766 words/s, in_qsiz

In [16]:
model.most_similar("man")

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-04-30 20:14:45,435 : INFO : precomputing L2-norms of word weight vectors


[(u'guy', 0.8088240027427673),
 (u'gentleman', 0.7590212821960449),
 (u'dude', 0.7199798822402954),
 (u'woman', 0.6940141916275024),
 (u'lady', 0.668986439704895),
 (u'girl', 0.625653088092804),
 (u'gentlemen', 0.6141798496246338),
 (u'gal', 0.5959008932113647),
 (u'gent', 0.5817946791648865),
 (u'bouncer', 0.5532838106155396)]

In [40]:
from __future__ import print_function

In [43]:
similar_list = model.most_similar(positive=["health"],topn=20)
for tupl in similar_list:
    print("'",tupl[0],"'",",",sep='',end='')

'medical','safety','healthcare','sanitation','wellbeing','hygiene','violations','wellness','billing','neurological','nutrition','financial','mental','inspectors','environmental','behavioral','illness','accounting','umc','hipaa',

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
